In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import scipy.optimize as scopt
import glob
import os

%matplotlib inline

## Setup

In [2]:
def load_json(file):
    '''
    Loads txt file and returns dataframe using the json dict
    '''
    data = {}
    with open(file, encoding='utf-8') as f:
        for line in f:
            d = str(line).replace("'", '"')
            data = json.loads(d)

    return pd.DataFrame(data)

In [3]:
os.chdir(os.getcwd()+'/rss-data')

## Retrieving and loading each trace

In [4]:
files = [file for file in glob.glob("*.txt")]
dfs = [load_json(file) for file in files]
data = pd.concat(dfs)
data['rss'] = pd.to_numeric(data['rss'])

## Separating traces by transmitter/MAC

In [5]:
set(data.mac)

{'8c:85:90:16:0a:a4',
 'ac:9e:17:7d:31:e8',
 'd8:c4:6a:50:e3:b1',
 'f8:cf:c5:97:e0:9e'}

In [6]:
t1 = data[data['mac'] == '8c:85:90:16:0a:a4']
t2 = data[data['mac'] == 'ac:9e:17:7d:31:e8']
t3 = data[data['mac'] == 'd8:c4:6a:50:e3:b1']
t4 = data[data['mac'] == 'f8:cf:c5:97:e0:9e']

## Defining functions for scipy.optimize.curve_fit

In [7]:
def localization(coords, c, g, x0, y0):
    x1, y1 = coords
    return c + g * np.log10(((x1-x0)**2 + (y1-y0)**2)**0.5)

In [8]:
def fit(tx):
    popt, pcov = scopt.curve_fit(localization,
                                 (tx.loc_x,tx.loc_y),
                                 tx.rss,
                                 maxfev=30000,
                                 bounds=((-60,2,-20,-20),(30,6,20,20)))
    return popt

## Fitting the data
The output follows the format [C, $\gamma$, x, y]. 

```x1, y1 = (6.8, 6.8)
x2, y2 = (-.87, 9.45)```


#### 8c:85:90:16:0a:a4

In [9]:
fit(t1)

array([-58.04559235,   6.        ,  -0.91649324,   7.18174916])

#### ac:9e:17:7d:31:e8

In [10]:
fit(t2)

array([-53.19515604,   6.        ,   6.48825705,   3.78141614])

#### d8:c4:6a:50:e3:b1

In [11]:
fit(t3)

array([-60.        ,   3.88950259,   6.41979309,   4.65179377])

#### f8:cf:c5:97:e0:9e

In [12]:
fit(t4)

array([-60.        ,   2.        ,   4.66652953,   6.78223933])